In [1]:
from keras.layers import Input, Reshape, Dropout, Dense, Flatten, BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model, load_model
from keras.optimizers import Adam
import numpy as np
from PIL import Image
import os

Using TensorFlow backend.
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [2]:
# THIS CODE WILL ONLY WORK IF YOU HAVE AN NVIDIA GRAPHICS CARD IN YOUR COMPUTER/LAPTOP
import tensorflow as tf
import keras
tf_config = tf.ConfigProto(allow_soft_placement=True)
tf_config.gpu_options.allow_growth = True
sess = tf.Session(config=tf_config) 
keras.backend.set_session(sess)

In [3]:
## image_resizer.py
# Defining an image size and image channel
# We are going to resize all our images to 128X128 size and since our images are colored images
# We are setting our image channels to 3 (RGB)

IMAGE_SIZE = 128
IMAGE_CHANNELS = 3
IMAGE_DIR = r'C:\Users\leont\Google Drive\CLASSES\ALK 2019-2020\Advanced Python\Students PROJECTS\High_R'
# Defining image dir path. Change this if you have different directory
images_path = IMAGE_DIR 

training_data = []

# Iterating over the images inside the directory and resizing them using
# Pillow's resize method.
print('resizing...')

for filename in os.listdir(images_path):
    path = os.path.join(images_path, filename)
    image = Image.open(path).resize((IMAGE_SIZE, IMAGE_SIZE), Image.ANTIALIAS)

    training_data.append(np.asarray(image))

training_data = np.reshape(
    training_data, (-1, IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS))
training_data = training_data / 127.5 - 1

print('saving file...')
np.save('hIGH_data.npy', training_data)

resizing...
saving file...


In [4]:
# Preview image Frame
PREVIEW_ROWS = 4
PREVIEW_COLS = 7
PREVIEW_MARGIN = 4
SAVE_FREQ = 100
# Size vector to generate images from
NOISE_SIZE = 100
# Configuration
EPOCHS = 10000 # number of iterations
BATCH_SIZE = 32
GENERATE_RES = 3
IMAGE_SIZE = 128 # rows/cols
IMAGE_CHANNELS = 3

In [5]:
training_data = np.load(os.path.join(r'C:\Users\leont\Google Drive\CLASSES\ALK 2019-2020\Advanced Python\Students PROJECTS\hIGH_data.npy'))

In [6]:
def build_discriminator(image_shape):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=3, strides=2,
    input_shape=image_shape, padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding='same'))
    model.add(ZeroPadding2D(padding=((0, 1), (0, 1))))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(256, kernel_size=3, strides=1, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(512, kernel_size=3, strides=1, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    input_image = Input(shape=image_shape)
    validity = model(input_image)
    return Model(input_image, validity)

In [7]:
def build_generator(noise_size, channels):
    model = Sequential()
    model.add(Dense(4 * 4 * 256, activation='relu',input_dim=noise_size))
    model.add(Reshape((4, 4, 256)))
    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation('relu'))
    model.add(UpSampling2D())
    model.add(Conv2D(256, kernel_size=3, padding='same'))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Activation('relu'))
    for i in range(GENERATE_RES):
         model.add(UpSampling2D())
         model.add(Conv2D(256, kernel_size=3, padding='same'))
         model.add(BatchNormalization(momentum=0.8))
         model.add(Activation('relu'))
    model.summary()
    model.add(Conv2D(channels, kernel_size=3, padding='same'))
    model.add(Activation('tanh'))
    input = Input(shape=(noise_size,))
    generated_image = model(input)
    
    return Model(input, generated_image)

In [8]:
def save_images(cnt, noise):
    image_array = np.full((
        PREVIEW_MARGIN + (PREVIEW_ROWS * (IMAGE_SIZE + PREVIEW_MARGIN)),
        PREVIEW_MARGIN + (PREVIEW_COLS * (IMAGE_SIZE + PREVIEW_MARGIN)), 3),
        255, dtype=np.uint8)
    generated_images = generator.predict(noise)
    generated_images = 0.5 * generated_images + 0.5
    image_count = 0
    for row in range(PREVIEW_ROWS):
            for col in range(PREVIEW_COLS):
                r = row * (IMAGE_SIZE + PREVIEW_MARGIN) + PREVIEW_MARGIN
                c = col * (IMAGE_SIZE + PREVIEW_MARGIN) + PREVIEW_MARGIN
                image_array[r:r + IMAGE_SIZE, c:c +
                            IMAGE_SIZE] = generated_images[image_count] * 255
                image_count += 1
    output_path = 'output'
    if not os.path.exists(output_path):
            os.makedirs(output_path)
    filename = os.path.join(output_path, f"trained-{cnt}.png")
    im = Image.fromarray(image_array)
    im.save(filename)

In [9]:
image_shape = (IMAGE_SIZE, IMAGE_SIZE, IMAGE_CHANNELS)
optimizer = Adam(1.5e-4, 0.5)
discriminator = build_discriminator(image_shape)
discriminator.compile(loss='binary_crossentropy',
optimizer=optimizer, metrics=['accuracy'])
generator = build_generator(NOISE_SIZE, IMAGE_CHANNELS)
random_input = Input(shape=(NOISE_SIZE,))
generated_image = generator(random_input)
discriminator.trainable = False
validity = discriminator(generated_image)
combined = Model(random_input, validity)
combined.compile(loss='binary_crossentropy',
optimizer=optimizer, metrics=['accuracy'])
y_real = np.ones((BATCH_SIZE, 1))
y_fake = np.zeros((BATCH_SIZE, 1))
fixed_noise = np.random.normal(0, 1, (PREVIEW_ROWS * PREVIEW_COLS, NOISE_SIZE))
cnt = 1
for epoch in range(EPOCHS):
    print('Start epoch:', epoch)
    idx = np.random.randint(0, training_data.shape[0], BATCH_SIZE)
    x_real = training_data[idx]

    noise= np.random.normal(0, 1, (BATCH_SIZE, NOISE_SIZE))
    x_fake = generator.predict(noise)
 
    discriminator_metric_real = discriminator.train_on_batch(x_real, y_real)
    discriminator_metric_generated = discriminator.train_on_batch(x_fake, y_fake)

    discriminator_metric = 0.5 * np.add(discriminator_metric_real, discriminator_metric_generated)
    generator_metric = combined.train_on_batch(noise, y_real)
    if epoch % SAVE_FREQ == 0:
        save_images(cnt, fixed_noise)
        cnt += 1

        print(f'{epoch} epoch, Discriminator accuracy: {100*  discriminator_metric[1]}, Generator accuracy: {100 * generator_metric[1]}')


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 4096)              413696    
_________________________________________________________________
reshape_1 (Reshape)          (None, 4, 4, 256)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 8, 8, 256)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 8, 8, 256)         590080    
_________________________________________________________________
batch_normalization_5 (Batch (None, 8, 8, 256)         1024      
_________________________________________________________________
activation_1 (Activation)    (None, 8, 8, 256)         0         
__________________________

C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'
C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


0 epoch, Discriminator accuracy: 29.6875, Generator accuracy: 100.0
Start epoch: 1


C:\ProgramData\Anaconda3\lib\site-packages\keras\engine\training.py:297: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


Start epoch: 2
Start epoch: 3
Start epoch: 4
Start epoch: 5
Start epoch: 6
Start epoch: 7
Start epoch: 8
Start epoch: 9
Start epoch: 10
Start epoch: 11
Start epoch: 12
Start epoch: 13
Start epoch: 14
Start epoch: 15
Start epoch: 16
Start epoch: 17
Start epoch: 18
Start epoch: 19
Start epoch: 20
Start epoch: 21
Start epoch: 22
Start epoch: 23
Start epoch: 24
Start epoch: 25
Start epoch: 26
Start epoch: 27
Start epoch: 28
Start epoch: 29
Start epoch: 30
Start epoch: 31
Start epoch: 32
Start epoch: 33
Start epoch: 34
Start epoch: 35
Start epoch: 36
Start epoch: 37
Start epoch: 38
Start epoch: 39
Start epoch: 40
Start epoch: 41
Start epoch: 42
Start epoch: 43
Start epoch: 44
Start epoch: 45
Start epoch: 46
Start epoch: 47
Start epoch: 48
Start epoch: 49
Start epoch: 50
Start epoch: 51
Start epoch: 52
Start epoch: 53
Start epoch: 54
Start epoch: 55
Start epoch: 56
Start epoch: 57
Start epoch: 58
Start epoch: 59
Start epoch: 60
Start epoch: 61
Start epoch: 62
Start epoch: 63
Start epoch: 64


Start epoch: 475
Start epoch: 476
Start epoch: 477
Start epoch: 478
Start epoch: 479
Start epoch: 480
Start epoch: 481
Start epoch: 482
Start epoch: 483
Start epoch: 484
Start epoch: 485
Start epoch: 486
Start epoch: 487
Start epoch: 488
Start epoch: 489
Start epoch: 490
Start epoch: 491
Start epoch: 492
Start epoch: 493
Start epoch: 494
Start epoch: 495
Start epoch: 496
Start epoch: 497
Start epoch: 498
Start epoch: 499
Start epoch: 500
500 epoch, Discriminator accuracy: 29.6875, Generator accuracy: 0.0
Start epoch: 501
Start epoch: 502
Start epoch: 503
Start epoch: 504
Start epoch: 505
Start epoch: 506
Start epoch: 507
Start epoch: 508
Start epoch: 509
Start epoch: 510
Start epoch: 511
Start epoch: 512
Start epoch: 513
Start epoch: 514
Start epoch: 515
Start epoch: 516
Start epoch: 517
Start epoch: 518
Start epoch: 519
Start epoch: 520
Start epoch: 521
Start epoch: 522
Start epoch: 523
Start epoch: 524
Start epoch: 525
Start epoch: 526
Start epoch: 527
Start epoch: 528
Start epoch: 5

Start epoch: 938
Start epoch: 939
Start epoch: 940
Start epoch: 941
Start epoch: 942
Start epoch: 943
Start epoch: 944
Start epoch: 945
Start epoch: 946
Start epoch: 947
Start epoch: 948
Start epoch: 949
Start epoch: 950
Start epoch: 951
Start epoch: 952
Start epoch: 953
Start epoch: 954
Start epoch: 955
Start epoch: 956
Start epoch: 957
Start epoch: 958
Start epoch: 959
Start epoch: 960
Start epoch: 961
Start epoch: 962
Start epoch: 963
Start epoch: 964
Start epoch: 965
Start epoch: 966
Start epoch: 967
Start epoch: 968
Start epoch: 969
Start epoch: 970
Start epoch: 971
Start epoch: 972
Start epoch: 973
Start epoch: 974
Start epoch: 975
Start epoch: 976
Start epoch: 977
Start epoch: 978
Start epoch: 979
Start epoch: 980
Start epoch: 981
Start epoch: 982
Start epoch: 983
Start epoch: 984
Start epoch: 985
Start epoch: 986
Start epoch: 987
Start epoch: 988
Start epoch: 989
Start epoch: 990
Start epoch: 991
Start epoch: 992
Start epoch: 993
Start epoch: 994
Start epoch: 995
Start epoch: 9

Start epoch: 1382
Start epoch: 1383
Start epoch: 1384
Start epoch: 1385
Start epoch: 1386
Start epoch: 1387
Start epoch: 1388
Start epoch: 1389
Start epoch: 1390
Start epoch: 1391
Start epoch: 1392
Start epoch: 1393
Start epoch: 1394
Start epoch: 1395
Start epoch: 1396
Start epoch: 1397
Start epoch: 1398
Start epoch: 1399
Start epoch: 1400
1400 epoch, Discriminator accuracy: 100.0, Generator accuracy: 0.0
Start epoch: 1401
Start epoch: 1402
Start epoch: 1403
Start epoch: 1404
Start epoch: 1405
Start epoch: 1406
Start epoch: 1407
Start epoch: 1408
Start epoch: 1409
Start epoch: 1410
Start epoch: 1411
Start epoch: 1412
Start epoch: 1413
Start epoch: 1414
Start epoch: 1415
Start epoch: 1416
Start epoch: 1417
Start epoch: 1418
Start epoch: 1419
Start epoch: 1420
Start epoch: 1421
Start epoch: 1422
Start epoch: 1423
Start epoch: 1424
Start epoch: 1425
Start epoch: 1426
Start epoch: 1427
Start epoch: 1428
Start epoch: 1429
Start epoch: 1430
Start epoch: 1431
Start epoch: 1432
Start epoch: 14

KeyboardInterrupt: 